In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import matplotlib.pyplot as plt
import numpy as np

from dataclasses import dataclass, field
from typing import List

In [7]:
x, y, h = 10, 20, 30

space = np.meshgrid(
    np.linspace(0, x, x),
    np.linspace(0, y, y),
    np.linspace(0, h, h),
    indexing='ij'
)

viewing_angle = np.array([0, 1, 0])  # looking forward

In [8]:
np.array(space).shape

(3, 10, 20, 30)

In [ ]:
@dataclass
class Camera:
    focal_length: float
    resolution: List[int, int]

    position: List[float, float, float]=field(default_factory=[0., 0., 0.])
    angle: List[float, float, float]=field(default_factory=[0., 1., 0.])


    # def __post_init__(self):
    #     self.c = self.a + self.b

    def _rotation_angle(self, std):
        cos = np.dot(std, self.angle)  # / length of angle, which is 1 as it is a unit vector
        sin = np.linalg.norm(np.cross(std, self.angle))

        return cos, sin

    def transform(self, 
        space,
        world_position, world_angle
    ):
        translation = self.position - world_position  # row vector

        assert (l := np.linalg.norm(self.angle)) != 0, f"value of camera's angle is weird"
        norm_angle = self.angle / l

        # rotational angles of 3 axis
        cos_x, sin_x = self._rotation_angle(np.array([1, 0, 0]))
        cos_y, sin_y = self._rotation_angle(np.array([0, 1, 0]))
        cos_h, sin_h = self._rotation_angle(np.array([0, 0, 1]))

        rotation_x = np.array([
            [cos_x, -sin_x, 0],
            [sin_x, cos_x, 0],
            [0, 0, 1],
        ])
        rotation_y = np.array([
            [1, 0, 0],
            [0, cos_y, sin_y],
            [0, sin_y, cos_y],
        ])
        rotation_h = np.array([
            [cos_h, 0, -sin_h],
            [0, 1, 0],
            [sin_h, 0, cos_h],
        ])
        rotation = rotation_x @ rotation_y @ rotation_h

        intrinsic = np.array([
            [self.focal_length, 0, 0, 0],
            [0, self.focal_length, 0, 0],
            [0, 0, 1, 0]
        ])

        abc = np.array([
            [1/self.resolution[0], 0, 1/self.resolution[0]/2],
            [0, 1/self.resolution[1], 1/self.resolution[1]/2],
            [0, 0, 1]
        ])

        # return transform_space

In [12]:
np.dot(
    np.array([2, 2, 1]),
    np.array([1, 0, 0]),
)

2

In [ ]:
points = np.ones((100, 3))
a = np.ones((4, 4))
b = np.ones()